In [8]:
def trading_strategy(data, x1, x2):
    buy_signals = [np.nan] * len(data)
    sell_signals = [np.nan] * len(data)
    profits = [0] * len(data)
    drawdowns = [0] * len(data)

    for i in range(1, len(data)):
        previous_close = data['Close'][i - 1]
        open_price = data['Open'][i]
        close_price = data['Close'][i]
        high_price = data['High'][i]

        # Buy condition
        if (open_price - previous_close) / previous_close >= x1 / 100:
            buy_signals[i] = open_price
            # Sell condition
            if (high_price - open_price) / open_price >= x2 / 100:
                sell_price = open_price * (1 + x2 / 100)
            else:
                sell_price = close_price
            sell_signals[i] = sell_price
            profit = (sell_price - open_price) / open_price
            profits[i] = profit
            # Drawdown calculation
            drawdown = (open_price - min(data['Low'][i:])) / open_price
            drawdowns[i] = drawdown

    data['Buy_Signal'] = buy_signals
    data['Sell_Signal'] = sell_signals
    data['Profit'] = profits
    data['Drawdown'] = drawdowns
    return data

# Now try running the strategy again
spy_data = fetch_data('SPY', '2020-01-01', '2020-12-31')
strategy_data = trading_strategy(spy_data, x1=1, x2=2)


[*********************100%%**********************]  1 of 1 completed


In [9]:
def summarize_strategy(data):
    # Calculate total number of trades
    total_trades = data['Buy_Signal'].count()

    # Calculate total profit
    total_profit = data['Profit'].sum()

    # Calculate average profit per trade
    average_profit = data['Profit'].mean()

    # Calculate maximum drawdown
    max_drawdown = data['Drawdown'].max()

    # Calculate success rate
    successful_trades = len(data[data['Profit'] > 0])
    success_rate = (successful_trades / total_trades) * 100 if total_trades > 0 else 0

    return {
        "Total Trades": total_trades,
        "Total Profit": total_profit,
        "Average Profit": average_profit,
        "Maximum Drawdown": max_drawdown,
        "Success Rate (%)": success_rate
    }

# Apply strategy to SPY
spy_data = fetch_data('SPY', '2020-01-01', '2020-12-31')
strategy_spy_data = trading_strategy(spy_data, x1=1, x2=2)

# Summarize the strategy
strategy_summary = summarize_strategy(strategy_spy_data)
for key, value in strategy_summary.items():
    print(f"{key}: {value:.2f}")



[*********************100%%**********************]  1 of 1 completed
Total Trades: 37.00
Total Profit: 0.13
Average Profit: 0.00
Maximum Drawdown: 0.33
Success Rate (%): 70.27


In [10]:
import numpy as np

def optimize_parameters(data, x1_range, x2_range):
    best_profit = -np.inf
    best_x1 = None
    best_x2 = None

    for x1 in x1_range:
        for x2 in x2_range:
            strategy_data = trading_strategy(data, x1, x2)
            total_profit = strategy_data['Profit'].sum()
            if total_profit > best_profit:
                best_profit = total_profit
                best_x1 = x1
                best_x2 = x2

    return best_x1, best_x2, best_profit

# Example usage
x1_range = np.arange(0.5, 2.0, 0.1)  # Adjust ranges as necessary
x2_range = np.arange(0.5, 3.0, 0.1)
best_x1, best_x2, best_profit = optimize_parameters(spy_data, x1_range, x2_range)

# Print the optimal parameters and the maximum profit
print(f"Optimal x1: {best_x1:.2f}%")
print(f"Optimal x2: {best_x2:.2f}%")
print(f"Maximum Profit: {best_profit:.2f}")

Optimal x1: 0.50%
Optimal x2: 2.90%
Maximum Profit: 0.21
The optimized parameters led to a profitable strategy.
These parameters should be further tested in different market conditions for robustness.


In [11]:
import pandas as pd

def compare_strategy_performance(spy_data, sss_data, x1, x2):
    # Apply strategy
    spy_strategy_data = trading_strategy(spy_data, x1, x2)
    sss_strategy_data = trading_strategy(sss_data, x1, x2)

    # Summary for SPY
    spy_summary = summarize_strategy(spy_strategy_data)
    print("SPY Strategy Performance:")
    for key, value in spy_summary.items():
        print(f"  {key}: {value:.2f}")

    # Summary for SSS
    sss_summary = summarize_strategy(sss_strategy_data)
    print("\nSSS Strategy Performance:")
    for key, value in sss_summary.items():
        print(f"  {key}: {value:.2f}")

    # Comparison
    print("\nComparison:")
    if spy_summary["Total Profit"] > sss_summary["Total Profit"]:
        print("  SPY yielded higher total profit.")
    else:
        print("  SSS yielded higher total profit.")

    if spy_summary["Success Rate (%)"] > sss_summary["Success Rate (%)"]:
        print("  SPY had a higher success rate.")
    else:
        print("  SSS had a higher success rate.")

# Simulate SSS data
np.random.seed(42)
dates = pd.date_range(start='2020-01-01', end='2020-12-31', freq='B')
sss_data = pd.DataFrame(index=dates)
sss_data['Open'] = np.random.uniform(20, 30, size=len(sss_data))
sss_data['High'] = sss_data['Open'] + np.random.uniform(0, 2, size=len(sss_data))
sss_data['Low'] = sss_data['Open'] - np.random.uniform(0, 2, size=len(sss_data))
sss_data['Close'] = np.random.uniform(sss_data['Low'], sss_data['High'], size=len(sss_data))

# Apply and compare strategy for SPY and SSS
compare_strategy_performance(spy_data, sss_data, best_x1, best_x2)



SPY Strategy Performance:
  Total Trades: 76.00
  Total Profit: 0.21
  Average Profit: 0.00
  Maximum Drawdown: 0.34
  Success Rate (%): 69.74

SSS Strategy Performance:
  Total Trades: 118.00
  Total Profit: 1.67
  Average Profit: 0.01
  Maximum Drawdown: 0.37
  Success Rate (%): 74.58

Comparison:
  SSS yielded higher total profit.
  SSS had a higher success rate.


In [12]:
def calculate_and_compare_max_drawdown(spy_data, sss_data, x1, x2):
    # Apply strategy to SPY and SSS
    spy_strategy_data = trading_strategy(spy_data, x1, x2)
    sss_strategy_data = trading_strategy(sss_data, x1, x2)

    # Calculate max drawdown for SPY and SSS
    max_drawdown_spy = calculate_max_drawdown(spy_strategy_data)
    max_drawdown_sss = calculate_max_drawdown(sss_strategy_data)

    # Print the maximum drawdowns
    print(f"Maximum Drawdown for SPY with optimal parameters (x1={x1}, x2={x2}): {max_drawdown_spy:.2f}")
    print(f"Maximum Drawdown for SSS with optimal parameters (x1={x1}, x2={x2}): {max_drawdown_sss:.2f}")

    # Comparison
    if max_drawdown_spy > max_drawdown_sss:
        print("SPY experienced a higher maximum drawdown than SSS.")
    else:
        print("SSS experienced a higher maximum drawdown than SPY.")

# Calculate and compare maximum drawdown
calculate_and_compare_max_drawdown(spy_data, sss_data, best_x1, best_x2)



Maximum Drawdown for SPY with optimal parameters (x1=0.5, x2=2.8999999999999995): 0.34
Maximum Drawdown for SSS with optimal parameters (x1=0.5, x2=2.8999999999999995): 0.37
SSS experienced a higher maximum drawdown than SPY.
